In [1]:
import pandas as pd
import numpy as np
import altair as alt

# Set up some global config and variables
alt.renderers.enable('default')
pd.options.mode.chained_assignment = None

# Read the raw data
df = pd.read_csv('jhu-daily-reports.csv')
samples = df[['Date', 'Country']].groupby('Date').Country.nunique()
days = samples[samples > 1].index.tolist()
df = df[df['Date'].isin(days)]

In [6]:
sl = df[df['Country'] == 'US'].groupby(['State', 'Date'], as_index=False).sum()
sl.Date = pd.to_datetime(sl.Date, format='%m-%d-%Y')
sl['Day'] = sl.Date.dt.day_name()
sl['Week'] = sl.Date.dt.week

dow = sl[sl['Week']>9]
dow = dow.groupby(['Week', 'Day'], as_index=False).sum()
line=alt.Chart(dow[dow['Day'].isin(['Monday', 'Wednesday', 'Friday'])]).mark_line().encode(
    alt.X('Week:N'),
    alt.Y('Confirmed_New:Q'),
    color='Day:N',
    tooltip=['Day', 'Week']
)
point=line.mark_circle(size=60)
(line+point).properties(
    width=1200,
    height=800
).interactive()

alt.LayerChart(...)

In [3]:
from ipywidgets import interact
from datetime import timedelta

@interact(window=(7, 49, 7))
def chart(window=7):
    mow = sl.groupby('Date', as_index=False).sum()[['Date', 'Confirmed_New']]
    for date in mow.Date:
        dl = sl[sl['Date'] == date].merge(sl[(sl['Date']<=date) & (sl['Date']>date+timedelta(days=-window))].groupby(['State'], as_index=False).max()[['State', 'Confirmed_New']], on='State')
        states = dl[(dl.Confirmed_New_x == dl.Confirmed_New_y) & (dl.Confirmed_New_x != 0)].State.values
        mow.loc[(mow['Date'] == date), 'Max'] = len(states)
        mow.loc[(mow['Date'] == date), 'States'] = str(states)

    bars = alt.Chart(mow).mark_bar(color='purple').encode(
        x='Date:T',
        y='Max:Q',
        tooltip=['Date', 'Max', 'States']
    )
    line = alt.Chart(mow).mark_line().encode(
        x='Date:T',
        y='Confirmed_New',
        tooltip=['Date', 'Confirmed_New']
    )
    points=line.mark_circle(size=60)
    reg = points.transform_regression('Date', 'Confirmed_New', method='poly', order=10).mark_line(strokeDash=[6,8], color='grey')
    nc = (line+points+reg)
    return (bars+nc).resolve_scale(y='independent').properties(
        width=1200,
        height=800
    ).interactive()

interactive(children=(IntSlider(value=7, description='window', max=49, min=7, step=7), Output()), _dom_classes…

In [7]:
from scipy import stats

np.seterr(all='ignore')
@interact(window=(7, 49, 7), value=['Confirmed_New', 'Deaths_New'])
def chart(window=14, value='Confirmed_New'):
    mow = sl.groupby('Date', as_index=False).sum()
    mow = mow[(mow['Confirmed_New'] > 0) & (mow['Date']!='2020-02-28')]
    for date in mow.Date:
        dl = mow[(mow['Date']<=date) & (mow['Date']>date-timedelta(days=window))]
        slope, intercept, r_value, p_value, std_err = stats.linregress(dl.Confirmed, dl[value])
        mow.loc[(mow['Date'] == date), 'Slope'] = slope

    bars = alt.Chart(mow).mark_bar(color='purple').encode(
        x='Date:T',
        y='Slope:Q',
        tooltip=['Date', 'Slope']
    )
    line = alt.Chart(mow).mark_line().encode(
        x='Date:T',
        y=value,
        tooltip=['Date', value]
    )
    points=line.mark_circle(size=60)
    reg = points.transform_regression('Date', value, method='poly', order=10).mark_line(strokeDash=[6,8], color='grey')
    nc = (line+points+reg)
    return (bars+nc).resolve_scale(y='independent').properties(
        width=1200,
        height=800
    ).interactive()

interactive(children=(IntSlider(value=14, description='window', max=49, min=7, step=7), Dropdown(description='…